# Parallel Cyclic Reduction

**Literatur**:

[1] Parallel Computers 2, Hockney & Jesshope, 1988, p. 475 - 483

## Parallel Cyclic Reduction (PCR)

Das **Parallel Cyclic Reduction (PCR)** Verfahren dient der Lösung linearer Gleichungssysteme mit tridiagonalen Matrizen. Im Gegensatz zum **Thomas-Algorithmus**, welcher aufgrund von Datenabhängigkeiten strikt seriell arbeiten muss, ermöglicht PCR eine effiziente Parallelisierung auf Kosten zusätzlicher Rechenoperationen. Dabei wird der bei der klassischen *Cyclic Reduction* auftretende Engpass (Reduktion der Parallelität in der Rückwärts-Einsetzung) vermieden.

Die Grundidee des PCR-Verfahrens besteht darin, Unbekannte mithilfe benachbarter Gleichungen sukzessive zu eliminieren, bis das System diagonalisiert ist und die Ergebnisse direkt abgelesen werden können. Hierfür werden in jedem Schritt Gleichungen miteinander gekoppelt. Diese Kopplung stellt sich im ersten Schritt wie folgt dar:

$$
\begin{alignat*}{5}
\color{teal} a_{i-1}x_{i-2} &\color{teal} + b_{i-1}x_{i-1} &&\color{teal} + c_{i-1}x_{i} && &&\color{teal} = k_{i-1} \\
               & \phantom{+} a_{i}x_{i-1}   &&+ \mathbf{b_{i}x_{i}}   &&+ c_{i}x_{i+1} &&= k_{i} \\
               &                 && \phantom{+} \color{violet} a_{i+1}x_{i} &&\color{violet} + b_{i+1}x_{i+1} &\color{violet} + c_{i+1}x_{i+2} &\color{violet} = k_{i+1}
\end{alignat*}
$$

Nach diesem ersten Reduktionsschritt hängt die Unbekannte $x_i$ nur noch von den weit entfernten Unbekannten $x_{i-2}$ und $x_{i+2}$ ab:

$$
\begin{align*}
    a^{(1)}_{i}\textcolor{teal}{x_{i-2}} + b^{(1)}_{i}\mathbf{x_{i}} + c^{(1)}_{i}\textcolor{violet}{x_{i+2}} = k^{(1)}_{i}
\end{align*}
$$

In jedem weiteren Schritt koppeln die Variablen mit jeweils weiter entfernt liegenden Nachbarn (Distanz $i=2^{k}$). Dieses Vorgehen wird für $k=1, 2, \dots, \lceil \log_2 n \rceil$ wiederholt, bis die gekoppelten Indizes außerhalb der Matrixgrenzen liegen. Übrig bleibt ein System, in dem nur noch die $x_i$ auf der Hauptdiagonalen verbleiben und somit gelöst sind.

Folgendes Beispiel soll dies an einer einfachen 4x4 Matrix visualisieren. Eine entsprechende prototypische Implementierung des Algorithmus befindet sich in pcr.py.

In [1]:
from pcr import solve, _alpha_gamma_calculation, _update_coefficients, diagonal, update_matrix
import numpy as np
import time

$$A =\begin{pmatrix}
4 & 8 & 0 & 0 \\
8 & 4 & 2 & 0 \\
0 & 2 & 8 & 4 \\
0 & 0 & 8 & 16
\end{pmatrix}, 
\quad
x = 
\begin{pmatrix}
1 \\ 2 \\ 3 \\ 4
\end{pmatrix},
y = A \cdot x =
\begin{pmatrix}
4 & 8 & 0 & 0 \\
8 & 4 & 2 & 0 \\
0 & 2 & 8 & 4 \\
0 & 0 & 8 & 16
\end{pmatrix}
\begin{pmatrix}
1 \\ 2 \\ 3 \\ 4
\end{pmatrix}
=
\begin{pmatrix}
20 \\ 22 \\ 44 \\ 88
\end{pmatrix}$$

Da die Matrix aus 4 Zeilen/Spalten besteht, werden in Summe $log_2(4) = 2$ Schritte des PCR-Verfahrens benötigt.


In [2]:
A = np.array([[4, 8, 0, 0], [8,4,2, 0], [0, 2,8,4], [0, 0, 8, 16]]).astype(float)
x = np.array([1, 2, 3, 4])
y = np.dot(A, x)

In [10]:
solve(A, y)

array([1., 2., 3., 4.])

### Level 1: 1. Step
Im ersten Schritt werden $\alpha$ und $\gamma$ bestimmt, welche im weiteren Verlauf für die Aktualisierung der Zeilenkoeffizienten $a_i, b_i, c_i, y_i$ benötigt werden. Da es sich ausschließlich um tridiagonale Matrizen handelt, wird in der entsprechenden Implementierung nur mit den entsprechenden Diagonalen gearbeitet. Dazu wird eine Klasse *Diagonals* verwendet, welche auch gleichzeitig die entsprechenden Randbedingungen bei Zugriffen außerhalb der Problemmatrix zurückgibt.

In [35]:
a = diagonal(np.concatenate((np.array([0], dtype=np.float64), get_diagonal(A, -1))), "left")
b = diagonal(get_diagonal(A, 0), "center")
c = diagonal(np.concatenate((get_diagonal(A, 1), np.array([0], dtype=np.float64))), "right")
y = diagonal(y)
# print("a = ", a)
# print("b = ", b)
# print("c = ", c)
# print("y = ", y)

In [30]:
c.diagonal

array([8., 2., 4., 0.])

In [31]:
_alpha_gamma_calculation(a, b, c, 1, 1) # alpha, gamma for row 1 and level 1

(-0.0, -2.0)

### Level 1: 2. Step
In diesem Schritt werden nun auf Grundlage von $\alpha$ und $\gamma$ die neuen Koeffizienten der jeweiligen Zeilen bestimmt. Hierbei handelt es sich um den ersten Entkopplungsschritt, wie bei der Aktualisierung der Matrix deutlich wird.

In [36]:
_update_coefficients(a, b, c, y, 1)

In [39]:
print("a_1 = ", a.diagonal)
print("b_1 = ", b.diagonal)
print("c_1 = ", c.diagonal)
print("y_1 = ", y.diagonal)

a_1 =  [-0. -0. -4. -2.]
b_1 =  [-12.  -12.5   5.   12. ]
c_1 =  [-4. -1. -0. -0.]
y_1 =  [-24. -29.  11.  44.]


In [40]:
A_new_1 = update_matrix(a, b, c, level=1)

In [41]:
A_new_1

array([[-12. ,   0. ,  -4. ,   0. ],
       [  0. , -12.5,   0. ,  -1. ],
       [ -4. ,   0. ,   5. ,   0. ],
       [  0. ,  -2. ,   0. ,  12. ]])

In der aktualisierten Matrix ist nun deutlich zu erkennen, dass $x_1$ mit $x_3$ und $x_2$ mit $x_4$ gekoppelt ist und durch den ersten Schritt in Zeile 1 die Abhängigkeit von $x_2$, in Zeile 2 die Abhängigkeit von $x_1$, in Zeile 3 die Abhängigkeit von $x_2$ sowie in Zeile 4 die Abhängigkeit von $x_3$ entfernt werden konnte. Im nächsten und finalen Schritt werden nun die noch übriggebliebenen Abhängigkeiten entfernt.

### Level 2: 1. & 2. Step

In [42]:
_update_coefficients(a, b, c, y, 2)

In [43]:
print("a_2 = ", a.diagonal)
print("b_2 = ", b.diagonal)
print("c_2 = ", c.diagonal)
print("y_2 = ", y.diagonal)

a_2 =  [0. 0. 0. 0.]
b_2 =  [-15.2        -12.66666667   6.33333333  12.16      ]
c_2 =  [-0. -0.  0.  0.]
y_2 =  [-15.2        -25.33333333  19.          48.64      ]


In [44]:
A_new_2 = update_matrix(a, b, c, level=2)

In [45]:
A_new_2

array([[-15.2       ,   0.        ,   0.        ,   0.        ],
       [  0.        , -12.66666667,   0.        ,   0.        ],
       [  0.        ,   0.        ,   6.33333333,   0.        ],
       [  0.        ,   0.        ,   0.        ,  12.16      ]])

In [46]:
y.diagonal

array([-15.2       , -25.33333333,  19.        ,  48.64      ])

Nun ist erkennbar, dass keine Kopplung mehr zwischen den Unbekannten entsteht und die Lösung direkt durch Division von $y$ und den Elementen der Hauptdiagonale gewonnen werden kann.

In [47]:
y.diagonal / A_new_2.diagonal(0)

array([1., 2., 3., 4.])

Die Schritte können in Kurzform durch die Solve-Methode aufgerufen werden:

## Weitere Beispiele

In [3]:
m_size = 40000
A = np.zeros((m_size,m_size))
x = np.arange(1, m_size+1)

x[7] = 900

a = diagonal(np.random.randint(1, 100, m_size).astype(np.float64)) 
b = diagonal(np.random.randint(1, 100, m_size).astype(np.float64)) 
c = diagonal(np.random.randint(1, 100, m_size).astype(np.float64))

A = update_matrix(a, b, c, 0)

y = A.dot(x)

# solve(A, y)

# pcr = PCR(A, y)

In [7]:
start = time.time()
solve(A, y)
end = time.time()

print("Time needed for pcr: ", end-start)

# start = time.time()
# np.linalg.solve(A, y)
# end = time.time()

# print("Time needed for numpy linalg solver: ", end-start)

Time needed for pcr:  0.1948544979095459
